In [3]:
from fastf1 import Cache

# Cache.enable_cache(r"C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\cache")

Cache.enable_cache(r'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\cache')


import pandas as pd
import numpy as np

import os,json

import fastf1

In [13]:
session = fastf1.get_event(2021,1)
session

RoundNumber                                                   1
Country                                                 Bahrain
Location                                                 Sakhir
OfficialEventName    FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021
EventDate                                   2021-03-28 00:00:00
EventName                                    Bahrain Grand Prix
EventFormat                                        conventional
Session1                                             Practice 1
Session1Date                          2021-03-26 14:30:00+03:00
Session1DateUtc                             2021-03-26 11:30:00
Session2                                             Practice 2
Session2Date                          2021-03-26 18:00:00+03:00
Session2DateUtc                             2021-03-26 15:00:00
Session3                                             Practice 3
Session3Date                          2021-03-27 15:00:00+03:00
Session3DateUtc                         

In [28]:
season = 2021
gp = 1
session_name = 'Race'

session = fastf1.get_session(season, gp, session_name)
session.load(telemetry=True, weather=True)



core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.7.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No ca

//FIELD MAPPING

In [29]:
meta = {}

meta['season'] = season
meta['gp'] = gp
meta['race_name'] = session.event.get('EventName', session.event.get('Event'))
meta['race_date'] = session.event.get('EventDate', None)
meta['session'] = 'Race'
meta['circuit_name'] = getattr(session,'track_name',None) or session.event.get('Circuit', None)
meta['laps_total_in_race'] = int(session.laps['LapNumber'].max())


    
meta


{'season': 2021,
 'gp': 1,
 'race_name': 'Bahrain Grand Prix',
 'race_date': Timestamp('2021-03-28 00:00:00'),
 'session': 'Race',
 'circuit_name': None,
 'laps_total_in_race': 56}

In [30]:
rows = []

for idx in session.laps.index:

    lap = session.laps.loc[idx]

    row = {}
    
    row['season'] = season
    row['round'] = gp
    row['session'] = session.name

    row['driver_name'] = lap.Driver
    row['driver_number'] = lap.DriverNumber
    row['team'] = lap.Team
    
    row['lap_number'] = lap.LapNumber
    row['lap_time'] = lap.LapTime.total_seconds() if pd.notna(lap.LapTime) else None

    row['sector1_time'] = lap.Sector1Time.total_seconds() if pd.notna(lap.Sector1Time) else None
    row['sector2_time'] = lap.Sector2Time.total_seconds() if pd.notna(lap.Sector2Time) else None
    row['sector3_time'] = lap.Sector3Time.total_seconds() if pd.notna(lap.Sector3Time) else None

    row['is_outlap'] = pd.notna(lap.PitOutTime)
    row['is_inlap'] = pd.notna(lap.PitInTime)
    
    row['position'] = lap.Position
    row['gap_to_leader'] = None
    row['speed_trap'] = lap.SpeedST if pd.notna(lap.SpeedST) else lap.SpeedI2

    row['compound'] = lap.Compound
    row['tyre_age'] = lap.TyreLife
    row['stint_number'] = lap.Stint

    try:
        w = lap.get_weather_data()
        if w is not None:
            row['air_temp'] = float(w.get('AirTemp', None))
            row['track_temp'] = float(w.get('TrackTemp', None))
            row['humidity'] = float(w.get('Humidity', None))
            row['pressure'] = float(w.get('Pressure', None))
            row['rainfall'] = bool(w.get('Rainfall', False))
            
            row['wind_speed'] = float(w.get('WindSpeed', None))
            row['wind_direction'] = float(w.get('WindDirection', None))
            row['has_weather'] = True
    except:
        row['air_temp'] = row['track_temp'] = row['humidity'] = None
        row['pressure'] = row['wind_speed'] = row['wind_direction'] = None
        row['rainfall'] = False
        row['has_weather'] = False
    
    
    
    try:
        tel = lap.telemetry

        row['avg_speed'] = tel['Speed'].mean() if 'Speed' in tel.columns else None
        row['max_speed'] = tel['Speed'].max() if 'Speed' in tel.columns else None

        row['avg_throttle'] = tel['Throttle'].mean() if 'Throttle' in tel.columns else None
        row['std_throttle'] = tel['Throttle'].std() if 'Throttle' in tel.columns else None

        row['avg_brake'] = tel['Brake'].mean() if 'Brake' in tel.columns else None
        row['std_brake'] = tel['Brake'].std() if 'Brake' in tel.columns else None

        row['max_rpm'] = tel['RPM'].max() if 'RPM' in tel.columns else None

        row['avg_gear'] = tel['nGear'].mean() if 'nGear' in tel.columns else None
        del tel

    except Exception as e:
        
        print(f'Telemetry error for lap idx {idx} (driver {lap.Driver}): {e}')
        
        row['avg_speed'] = None  
        row['max_speed'] = None 

        row['avg_throttle'] = None
        row['std_throttle'] = None

        row['avg_brake'] = None
        row['std_brake'] = None

        row['avg_steering'] = None
        row['max_rpm'] = None

        row['avg_gear'] = None
        
    row.update(meta)
    rows.append(row) 
    
    

rows


[{'season': 2021,
  'round': 1,
  'session': 'Race',
  'driver_name': 'HAM',
  'driver_number': '44',
  'team': 'Mercedes',
  'lap_number': np.float64(1.0),
  'lap_time': 119.538,
  'sector1_time': None,
  'sector2_time': 54.367,
  'sector3_time': 32.463,
  'is_outlap': False,
  'is_inlap': False,
  'position': np.float64(2.0),
  'gap_to_leader': None,
  'speed_trap': np.float64(258.0),
  'compound': 'MEDIUM',
  'tyre_age': np.float64(4.0),
  'stint_number': np.float64(1.0),
  'air_temp': 20.5,
  'track_temp': 28.1,
  'humidity': 53.7,
  'pressure': 1014.6,
  'rainfall': False,
  'wind_speed': 1.3,
  'wind_direction': 10.0,
  'has_weather': True,
  'avg_speed': np.float64(155.12294762489933),
  'max_speed': np.float64(293.0),
  'avg_throttle': np.float64(51.04205654342345),
  'std_throttle': np.float64(41.16964022600254),
  'avg_brake': np.float64(0.23946360153256704),
  'std_brake': np.float64(0.4269606510246044),
  'max_rpm': np.float64(12883.0),
  'avg_gear': np.float64(3.9291187739

In [31]:
print(rows)
df = pd.DataFrame(rows)


[{'season': 2021, 'round': 1, 'session': 'Race', 'driver_name': 'HAM', 'driver_number': '44', 'team': 'Mercedes', 'lap_number': np.float64(1.0), 'lap_time': 119.538, 'sector1_time': None, 'sector2_time': 54.367, 'sector3_time': 32.463, 'is_outlap': False, 'is_inlap': False, 'position': np.float64(2.0), 'gap_to_leader': None, 'speed_trap': np.float64(258.0), 'compound': 'MEDIUM', 'tyre_age': np.float64(4.0), 'stint_number': np.float64(1.0), 'air_temp': 20.5, 'track_temp': 28.1, 'humidity': 53.7, 'pressure': 1014.6, 'rainfall': False, 'wind_speed': 1.3, 'wind_direction': 10.0, 'has_weather': True, 'avg_speed': np.float64(155.12294762489933), 'max_speed': np.float64(293.0), 'avg_throttle': np.float64(51.04205654342345), 'std_throttle': np.float64(41.16964022600254), 'avg_brake': np.float64(0.23946360153256704), 'std_brake': np.float64(0.4269606510246044), 'max_rpm': np.float64(12883.0), 'avg_gear': np.float64(3.92911877394636), 'gp': 1, 'race_name': 'Bahrain Grand Prix', 'race_date': Time

CheckPoint - Converting raw data to JSON and CSV to work on.

In [44]:
os.makedirs(r'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\raw', exist_ok=True)

raw_json_path = fr'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\raw\season_{season}_round_{gp}_raw.json'
with open(raw_json_path, 'w', encoding='utf8') as f:
    json.dump(rows,f,default=str, indent=2)
    
raw_csv_path = fr'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\raw\season_{season}_round_{gp}_raw.csv'
df.to_csv(raw_csv_path, index=False)

print('Saved raw JSON: ', raw_json_path)
print('Saved raw CSV: ', raw_csv_path)



Saved raw JSON:  C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\raw\season_2021_round_1_raw.json
Saved raw CSV:  C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\raw\season_2021_round_1_raw.csv


In [33]:
df_work = df.copy() ## data frame to work on

In [34]:
df_work = df_work.sort_values(by=['driver_name', 'lap_number']).reset_index(drop=True)
df_work['lap_time_delta'] = np.nan

# ----------
# Strategy 1 - Strict Delta

for driver, g in df_work.groupby('driver_name', sort=False):
    valid_idx = g[g['lap_time'].notna()].index
    
    if(len(valid_idx) <= 1) : continue
    
    delta = df_work.loc[valid_idx, 'lap_time'].diff().values
    df_work.loc[valid_idx, 'lap_time_delta'] = delta
    
df_work.loc[df_work['is_inlap'] | df_work['is_outlap'], 'lap_time_delta'] = np.nan
print('Total Count: ')
print('Total: ', df_work['lap_time_delta'].count())
print('Valid: ', df_work['lap_time_delta'].count()-df_work['lap_time_delta'].isnull().sum())
print('Null: ', df_work['lap_time_delta'].isnull().sum())





Total Count: 
Total:  916
Valid:  805
Null:  111


In [35]:
df_work['lap_time_best_on_tyre'] = False

df_valid_lap_times = df_work.dropna(subset=['lap_time'])

groups = df_valid_lap_times.groupby(['driver_name','stint_number'])
df_work.loc[
    groups['lap_time'].idxmin(),
    'lap_time_best_on_tyre'
 ] = True




Creating the clean JSON and CSV's

In [43]:
os.makedirs(r'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\clean', exist_ok=True)

clean_csv_path = fr'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\clean\season_{season}_round_{gp}_laps_clean.csv'
clean_json_path = fr'C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\clean\season_{season}_round_{gp}_laps_clean.json'

df_work.to_csv(clean_csv_path, index = False)

records = df_work.to_dict(orient='records')

with open(clean_json_path, 'w', encoding='utf8') as f:
    json.dump(records, f, default=str, indent=2)
    
print('Saved CSV: ',clean_csv_path,'\n', 'Saved JSON: ',clean_json_path)



Saved CSV:  C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\clean\season_2021_round_1_laps_clean.csv 
 Saved JSON:  C:\Users\ASUS\Desktop\F1 Predictions & Visualizations\F1-ML-Project\data\clean\season_2021_round_1_laps_clean.json


In [46]:
schedule = fastf1.get_event_schedule(2021)
schedule.head()

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Bahrain,FORMULA 1 ARAMCO PRE-SEASON TESTING 2021,2021-03-14,Pre-Season Test,testing,Practice 1,2021-03-12 10:00:00+03:00,2021-03-12 07:00:00,...,Practice 3,2021-03-14 10:00:00+03:00,2021-03-14 07:00:00,None,NaT,NaT,None,NaT,NaT,False
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26 14:30:00+03:00,2021-03-26 11:30:00,...,Practice 3,2021-03-27 15:00:00+03:00,2021-03-27 12:00:00,Qualifying,2021-03-27 18:00:00+03:00,2021-03-27 15:00:00,Race,2021-03-28 18:00:00+03:00,2021-03-28 15:00:00,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16 11:00:00+02:00,2021-04-16 09:00:00,...,Practice 3,2021-04-17 11:00:00+02:00,2021-04-17 09:00:00,Qualifying,2021-04-17 14:00:00+02:00,2021-04-17 12:00:00,Race,2021-04-18 15:00:00+02:00,2021-04-18 13:00:00,True
3,3,Portugal,Portimão,FORMULA 1 HEINEKEN GRANDE PRÉMIO DE PORTUGAL 2021,2021-05-02,Portuguese Grand Prix,conventional,Practice 1,2021-04-30 11:30:00+01:00,2021-04-30 10:30:00,...,Practice 3,2021-05-01 12:00:00+01:00,2021-05-01 11:00:00,Qualifying,2021-05-01 15:00:00+01:00,2021-05-01 14:00:00,Race,2021-05-02 15:00:00+01:00,2021-05-02 14:00:00,True
4,4,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2021,2021-05-09,Spanish Grand Prix,conventional,Practice 1,2021-05-07 11:30:00+02:00,2021-05-07 09:30:00,...,Practice 3,2021-05-08 12:00:00+02:00,2021-05-08 10:00:00,Qualifying,2021-05-08 15:00:00+02:00,2021-05-08 13:00:00,Race,2021-05-09 15:00:00+02:00,2021-05-09 13:00:00,True
